In [1]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv
from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:

import random
import time
from datasets import load_dataset, Dataset
import json
from openai.types.chat.chat_completion import ChatCompletion
import os
from transformers.utils.logging import disable_progress_bar, enable_progress_bar


default_or_options = {
    "max_tokens": 8191,

    "extra_body": {
        "provider": {
            "ignore": [
                "Hyperbolic"
            ],
            "require_parameters": True,
        },
    }
}
domain_events = load_dataset("lucaelin/generic_domain_events_v1", split="train")
domain_action_calls = load_dataset("lucaelin/generic_domain_action_dialogs_v1", split="train")
domain_actions = load_dataset("lucaelin/generic_domain_actions_v1", split="train")

def clean_tool_id(tool_id):
    return tool_id.replace(" ", "").replace('_', '').replace('-', '')[0:9]

def filter_metadata(dialog):
    dialog = json.loads(json.dumps(dialog))
    for msg in dialog:
        if "_metadata" in msg:
            del msg["_metadata"]
        if not 'content' in msg:
            msg['content'] = ''
        if 'tool_call_id' in msg:
            msg['tool_call_id'] = clean_tool_id(msg['tool_call_id'])
        if 'tool_calls' in msg:
            for tool_call in msg['tool_calls']:
                if 'id' in tool_call:
                    tool_call['id'] = clean_tool_id(tool_call['id'])
    
    return dialog

def generate_covas_commentary(entry):
    results = []

    available_tool_models = [
        #'anthropic/claude-3.5-sonnet-20240620',
        #'openai/chatgpt-4o-latest',
        #'openai/gpt-4o-mini-2024-07-18',
        "mistralai/mistral-nemo",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mistral-large"
    ]
    available_text_models = [
        #'anthropic/claude-3.5-sonnet-20240620',
        #'openai/chatgpt-4o-latest',
        #'openai/gpt-4o-mini-2024-07-18',
        'nousresearch/hermes-3-llama-3.1-70b',
        'nousresearch/hermes-3-llama-3.1-405b',
        'meta-llama/llama-3.1-405b-instruct',
        'meta-llama/llama-3.1-70b-instruct',
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mistral-large"
    ]


    disable_progress_bar()
    related_domain_events = domain_events.filter(lambda x: x["profession"] == entry["profession"]).to_list()
    if not related_domain_events:
        return []
    related_domain_actions = domain_actions.filter(lambda x: x["profession"] == entry["profession"]).to_list()
    if not related_domain_actions:
        return []
    related_domain_action_calls = domain_action_calls.filter(lambda x: x["profession"] == entry["profession"] and x["process"] == entry["process"] and len(x["dialog"])<15000).to_list()
    enable_progress_bar()
    tools = []

    for action in related_domain_actions:
        if len(action['parameter_schema']) < 10:
            continue
        name_lower = action["name"].lower()
        name_capitalized = ' '.join([word.lower().capitalize() for word in action["name"].split(' ')])
        action_names = [
            name_lower.replace(" ", "_"),
            name_lower.replace(" ", ""),
            name_lower.replace(" ", "-"),
            name_capitalized.replace(" ", "_"),
            name_capitalized.replace(" ", ""),
            name_capitalized.replace(" ", "-"),
        ]

        model = random.choice(available_tool_models)
        function_name = random.choice(action_names)
        function = {
            "type": "function",
            "function": {
                "name": function_name,
                "description": action["description"],
                "parameters": json.loads(action['parameter_schema']),
            },
        }
        tools.append(function)
    
    #print ("tools ", tools)

    user_name = random.choice(random_names)

    messages = [{
        "role": "system",
        "content": f"Let's roleplay as a {entry['profession']}.\n"
            "I will provide events in parentheses; do not create new ones. " +
            "Do not hallucinate any information that is not given to you. Do not use markdown in your responses. " +
            f"I am {user_name}, a professional {entry['profession']}. You are my digital assistant, sometimes also addressed as Computer. "+
            "You possess extensive knowledge and can provide detailed and accurate information on a wide range of topics. " +
            "Reply within one sentence, acknowledge orders, mention status/location only if relevant or asked, and don't end with a question."+
            "Guide and support me with witty commentary and humorous observations."

    }, {
        "role": "user",
        "_metadata": {
            "type": "situation",
            "situation": json.loads(entry['situation_json']),
        },
        "content": f"(Current situation: {entry['situation_json']})"
    }]

    def generate_commentary(dialog):
        model = random.choice(available_text_models)
        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=filter_metadata(dialog),
            #tools=tools,
            #tool_choice="required",
            **default_or_options,
        )
        comment = openrouter_response.choices[0].message.content.strip()
        if comment.startswith("Computer: "):
            comment = comment[len("Computer: "):]
        if comment.startswith('"'):
            comment = comment[1:-1]

        return [{
            "role": "assistant",
            "_metadata": {
                "type": "comment",
            },
            "content": comment
        }]

    def generate_event(dialog):
        model = "mistralai/mistral-nemo"
        event_names = [event["name"] for event in related_domain_events]
        random.shuffle(event_names)
        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=filter_metadata(dialog)+[{
                "role": "user",
                "content": f"Continue the roleplay by selecting one of the available events, that best fits the current situation. Use the select_next_event tool to choose an event."
            }],
            tools=[{
                "type": "function",
                "function": {
                    "name": "select_next_event",
                    "description": "Select the next event to happen in the roleplay.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "next_event": {
                                "type": "string",
                                "enum": event_names,
                            },
                        },
                    },
                },
            }],
            tool_choice="required",
            **default_or_options,
        )
        #print(dialog)
        #print(openrouter_response)
        event_name = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)["next_event"]
        event_definitions = [event for event in related_domain_events if event["name"] == event_name and len(event["schema"]) > 10]
        if not event_definitions:
            return []
        event_definition = event_definitions[0]
        
        model = random.choice(available_tool_models)
        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=filter_metadata(dialog)+[{
                "role": "user",
                "content": f"Continue the roleplay by creatily inventing additional content for the event {event_name} that matches the previous discussion, but may also introduce new entities. Use the generate_next_event tool to create the event."
            }],
            tools=[{
                "type": "function",
                "function": {
                    "name": "generate_next_event",
                    "description": "Generate the next event to happen in the roleplay.",
                    "parameters": json.loads(event_definition["schema"]),
                },
            }],
            tool_choice="required",
            **default_or_options,
        )
        #print(dialog)
        #print(openrouter_response)
        event_content = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)

        return [{
            "role": "user",
            "_metadata": {
                "type": "event",
                "name": event_name,
                "content": event_content,
                "definition": event_definition,
            },
            "content": f"({event_name}, details: {json.dumps(event_content)})"
        }]
    
    retries = 0
    runs = 0
    while len(filter_metadata(messages)) < 40 or (messages and messages[-1]["role"] != "assistant"):
        runs += 1
        if runs > 80:
            raise Exception("Too many runs")
        #print("tokens",len(json.dumps(filter_metadata(messages))) / 4)
        try:
            event = generate_event(messages)
            #print("event ", event)
            messages = messages + event
            if event and event[0]['_metadata']['definition']['severity']=='high' or random.random() < 0.2:
                comment = generate_commentary(messages)
                #print("comment ", comment)
                messages = messages + comment
            if related_domain_action_calls and random.random() > 0.1:
                tool_call = random.choice(related_domain_action_calls)
                related_domain_action_calls.remove(tool_call)
                tool_call_dialog = json.loads(tool_call["dialog"])
                #print("tool ", tool_call_dialog)
                messages = messages + tool_call_dialog
            retries = 0
        except Exception as e:
            retries += 1
            if retries > 10:
                raise Exception("Too many retries")
            #print("ERROR ", e)
            #traceback.print_exc()
            time.sleep(1)
            continue
        
    if related_domain_action_calls:
        tool_call = random.choice(related_domain_action_calls)
        related_domain_action_calls.remove(tool_call)
        tool_call_dialog = json.loads(tool_call["dialog"])
        #print("tool ", tool_call_dialog)
        messages = messages + tool_call_dialog

    results.append({
        "profession": entry["profession"],
        "process": entry["process"],
        "tools": tools,
        "messages": messages,
    })
    
    return results


situations = load_dataset("lucaelin/domain_situations_v1", split="train")
covas_commentary = situations.map(
    sane_batch_map(generate_covas_commentary, {"profession": [], "process": [], "tools": [], "messages": []}), 
    batched=True, 
    batch_size=1, 
    num_proc=20, 
    remove_columns=situations.column_names
)
#covas_commentary.sort(["profession", "process"]).to_list()
covas_commentary.sort(["profession", "process"]).push_to_hub("lucaelin/generic_covas_commentary")

Map (num_proc=20):  24%|██▍       | 174/723 [22:31<1:13:36,  8.04s/ examples]

{'profession': 'Compliance Officer', 'process': 'Risk Assessment', 'situation': "The process of Risk Assessment involves identifying, analyzing, and evaluating potential risks that could negatively impact the operations and objectives of an organization. The goal is to determine the likelihood and impact of these risks and implement appropriate measures to mitigate them. This process is crucial for regulatory compliance and maintaining a safe and efficient work environment. The desired outcome is a comprehensive risk management plan that can guide decision-making and help protect the organization's assets and reputation.", 'situation_json': '{"details": {"process_goal": "Determine likelihood and impact of risks and implement mitigation measures", "desired_outcome": "Comprehensive risk management plan", "importance": "Crucial for regulatory compliance and safe work environment", "specific_details": {"assets_impacted": "Organization\'s assets and reputation", "risk_analysis_steps": ["Ide

Filter: 100%|██████████| 645/645 [00:00<00:00, 77757.13 examples/s]


{'profession': 'Bookkeeping, Accounting, & Audit Clerk', 'process': 'Record Financial Transactions', 'situation': 'In the quiet and focused workspace of the accounting department, surrounded by the hum of the office environment, a Bookkeeping, Accounting, & Audit Clerk is immersed in the ongoing daily task of recording financial transactions. The clerk sits at a spacious desk with a well-maintained computer in front of them, a calculator within reach, and filing cabinets lined up against the wall. On one side of the desk is a neat stack of receipts, invoices, and bank statements, while on the other side is a printer, ready to churn out reports and financial documents. The room is free from distracting noise, allowing the clerk to concentrate on maintaining accurate financial records with utmost precision. The clerk is recording daily financial transactions, including income, expenses, accounts payable, and receivable, all with a sense of purpose and responsibility. Nearby, an accountan

Filter: 100%|██████████| 645/645 [00:00<00:00, 87562.34 examples/s]


...

{'profession': 'Physical Therapist Assistant', 'process': 'Treatment Plan Implementation', 'situation': "In the bustling, well-lit treatment room of a modern clinic, a Physical Therapist Assistant (PTA) is meticulously implementing a treatment plan for an elderly patient recovering from a hip replacement. The room is a symphony of equipment, precision, and care: a height-adjustable, navy blue therapy table takes center stage, its padded surface comforting and supportive, while an array of exercise equipment—resistance bands, weights, balance boards, and stability balls—lies neatly organized on shelves. A nearby computer, adorned with a quietly humming monitor, displays detailed patient charts and progress notes through the documentation software, its keyboard awaiting the PTA's frequent updates. The PTA's toolkit is extensive, with assistive devices like crutches, canes, and walkers standing ready in the corner, and therapeutic modalities such as thermotherapy units, cryotherapy units,

Map (num_proc=20): 100%|█████████▉| 722/723 [5:12:29<04:19, 259.71s/ examples]

{'profession': 'Security Guard', 'process': 'CCTV Monitoring', 'situation': 'In the dimly lit, cubical security office, a lone security guard is stationed. Four large CCTV monitors sit on the desk in front of them, each displaying a live feed from a different surveillance camera installed throughout the building. The guard wears a crisp, dark blue uniform and a radio is clipped to their belt. A notepad and pen sit close at hand, ready for documenting any incidents that may occur during their shift. The building is quiet, with only the hum of the monitors and the occasional chatter from the security checkpoint breaking the silence. The guard watches the screens intently, scanning each for any signs of suspicious activity. Their eyes flick between the different feeds, constantly vigilant. The guard is there for the entirety of their shift, their presence a reassuring symbol of safety and security for all within the building.', 'situation_json': '{"office_lighting": "dim", "office_descrip

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lucaelin/generic_covas_commentary/commit/6a5d2c2b9237860b7055e02b8ec8d24fb6051f50', commit_message='Upload dataset', commit_description='', oid='6a5d2c2b9237860b7055e02b8ec8d24fb6051f50', pr_url=None, pr_revision=None, pr_num=None)